In [1]:
# 1. INSTALL DEPENDENCIES
!pip install Flask pyngrok mistralai scikit-learn pandas joblib imblearn sentence-transformers wikipedia xgboost

In [2]:
# 2. KODE UTAMA (jalankan di sel kedua)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, learning_curve
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
import joblib
import os
import json
from flask import Flask, request, jsonify
import wikipedia
from sentence_transformers import SentenceTransformer, util
from typing import List, Dict, Optional
import re
import html
import urllib.parse
import threading
import time
from pyngrok import ngrok, conf
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from mistralai import Mistral
import textwrap
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Konfigurasi untuk Model 1
MODEL_NAME_M1 = 'paraphrase-multilingual-MiniLM-L12-v2'
WIKIPEDIA_LANG = "id"
NUM_SENTENCES_FOR_SUMMARY = 10
NUM_ARTICLES_TO_FETCH_PER_QUERY = 3
MIN_CONTENT_LENGTH_FOR_VALID_ARTICLE = 250
SIMILARITY_THRESHOLD = 0.4

# Konfigurasi untuk Model 2
MODEL_FILE_M2 = 'user_understanding_classifier.joblib'
DATA_FILE_M2 = '/content/rekomind_answer_history (4).csv'
df_raw = pd.read_csv(DATA_FILE_M2)

# Variabel global untuk model yang dimuat
_sentence_model: Optional[SentenceTransformer] = None
# _classifier_artifacts: Optional[dict] = None
_classifier_artifacts: Dict = {}

# **MODEL 1 Rekomendasi Bacaan**

In [4]:
def _load_sentence_model_once_colab():
    """Memuat model Sentence Transformer sekali saja."""
    global _sentence_model
    if _sentence_model is None:
        try:
            print(f"MODEL 1: Memuat model Sentence Transformer: '{MODEL_NAME_M1}'...")
            _sentence_model = SentenceTransformer(MODEL_NAME_M1)
            print(f"MODEL 1: Model Sentence Transformer '{MODEL_NAME_M1}' berhasil dimuat.")
        except Exception as e:
            print(f"MODEL 1: GAGAL memuat model Sentence Transformer '{MODEL_NAME_M1}': {e}")
            _sentence_model = None
    return _sentence_model

In [5]:
def _decode_and_clean_text_colab(text: str) -> str:
    """Membersihkan teks dari HTML, URL encoding, dan artefak lainnya."""
    if not text: return ""
    decoded_text = text
    decoded_text = urllib.parse.unquote(decoded_text)
    decoded_text = html.unescape(decoded_text)
    decoded_text = re.sub(r'==.*?==', '', decoded_text) # Hapus judul bagian wikipedia
    decoded_text = re.sub(r'\n', ' ', decoded_text) # Ganti baris baru dengan spasi
    decoded_text = re.sub(r'\s\s+', ' ', decoded_text) # Hapus spasi berlebih
    return decoded_text.strip()

In [6]:
def _fetch_wikipedia_articles_for_search_keyword_colab(search_keyword: str) -> List[Dict]:
    """Mencari beberapa artikel relevan dari Wikipedia."""
    print(f"  MODEL 1: Mencari artikel Wikipedia untuk keyword: '{search_keyword}'")
    wikipedia.set_lang(WIKIPEDIA_LANG)
    articles_info = []
    try:
        search_results = wikipedia.search(search_keyword, results=NUM_ARTICLES_TO_FETCH_PER_QUERY)
        if not search_results:
            print(f"    MODEL 1: Tidak ada hasil pencarian Wikipedia untuk '{search_keyword}'.")
            return []

        for title in search_results:
            try:
                page = wikipedia.page(title, auto_suggest=True, redirect=True)
                content = _decode_and_clean_text_colab(page.content)
                if len(content) >= MIN_CONTENT_LENGTH_FOR_VALID_ARTICLE:
                    articles_info.append({
                        "judul": page.title, "konten": content, "url": page.url,
                    })
                    print(f"    -> MODEL 1: Artikel '{page.title}' diproses (konten: {len(content)} chars).")
            except Exception as e_page:
                print(f"    MODEL 1 Error saat mengambil halaman '{title}': {e_page}")
    except Exception as e_search:
        print(f"  MODEL 1 Error saat wikipedia.search('{search_keyword}'): {e_search}")
    return articles_info

In [7]:
def get_dynamic_content_recommendations_colab(query_text_for_embedding: str, search_keyword_for_wikipedia: str) -> List[Dict]:
    """Menghasilkan rekomendasi konten berdasarkan pencarian dan kemiripan."""
    if _sentence_model is None: return [{"error": "Model AI (Sentence Transformer) Belum Siap"}]

    dynamic_corpus_articles = _fetch_wikipedia_articles_for_search_keyword_colab(search_keyword_for_wikipedia)
    if not dynamic_corpus_articles: return []

    all_sentences = []
    article_map = []
    for i, article in enumerate(dynamic_corpus_articles):
        sentences = article['konten'].split('. ')
        all_sentences.extend(sentences)
        article_map.extend([i] * len(sentences))

    query_embedding = _sentence_model.encode(query_text_for_embedding)
    corpus_embeddings = _sentence_model.encode(all_sentences)
    cosine_scores = util.cos_sim(query_embedding, corpus_embeddings)[0].numpy()

    top_results_indices = np.argsort(-cosine_scores)[:NUM_SENTENCES_FOR_SUMMARY]

    best_article_index = article_map[top_results_indices[0]]
    best_article = dynamic_corpus_articles[best_article_index]

    summary_sentences = [all_sentences[i] for i in top_results_indices if article_map[i] == best_article_index]
    summary = '. '.join(summary_sentences)

    if not summary: summary = all_sentences[top_results_indices[0]]

    return [{"judul": best_article['judul'], "summary": summary, "url": best_article['url'], "source": "Wikipedia (Dinamis)"}]

# **MODEL 2 HYBRID (Klasifikasi + LLM)**

In [8]:
# Di bagian atas skrip Anda, setelah deklarasi _classifier_artifacts
# _classifier_artifacts: Dict = {}

def _load_classifier_artifacts_once_colab():
    """Memuat semua artefak yang diperlukan untuk Model 2 (Ensemble)."""
    global _classifier_artifacts
    if not _classifier_artifacts: # Hanya muat jika kamus kosong
        try:
            print("MODEL 2: Memuat artefak untuk model Ensemble...")
            # Sesuaikan path jika Anda menyimpannya di Google Drive
            model_path = "ensemble_model_reko.pkl"
            scaler_path = "scaler_enhanced.pkl"
            encoder_path = "category_encoder.pkl"

            _classifier_artifacts["model"] = joblib.load(model_path)
            _classifier_artifacts["scaler"] = joblib.load(scaler_path)
            _classifier_artifacts["category_encoder"] = joblib.load(encoder_path)

            print("✅ MODEL 2: Artefak Ensemble (model, scaler, encoder) berhasil dimuat.")
        except Exception as e:
            print(f"GAGAL memuat artefak Model 2: {e}")
            _classifier_artifacts = {} # Kosongkan jika gagal
    return _classifier_artifacts

In [9]:
def get_holistic_hybrid_analysis_from_data(performance_list: List[Dict]) -> str:
    """
    Menganalisis performa pengguna menggunakan model Ensemble (RF+XGB+SVM)
    dan menghasilkan ringkasan dengan LLM.
    """
    print("\n--- [HYBRID MODEL V2] Memulai Analisis dengan Ensemble Model ---")

    if not _classifier_artifacts or "model" not in _classifier_artifacts:
        return "Error: Model klasifikasi (Ensemble) belum siap."
    if not performance_list:
        return "Tidak ada data performa yang diterima."

    try:
        df = pd.DataFrame(performance_list)

        # --- LANGKAH 1: REPLIKASI FEATURE ENGINEERING ---
        # Pastikan kolom-kolom yang diperlukan ada
        required_cols = {'category_name', 'total_questions', 'correct_count', 'difficulty_score'}
        if not required_cols.issubset(df.columns):
             # Buat kolom dummy jika tidak ada, meskipun idealnya data input harus lengkap
            if 'difficulty_score' not in df.columns: df['difficulty_score'] = 2.0
            if 'correct_count' not in df.columns: df['correct_count'] = df['accuracy'] * df['total_questions']


        # Buat fitur-fitur baru persis seperti saat training
        df['correct_ratio'] = df['correct_count'] / df['total_questions']
        df['interaction'] = df['total_questions'] * df['difficulty_score']
        df['difficulty_correct_ratio'] = df['difficulty_score'] * df['correct_ratio']

        # Ganti NaN yang mungkin muncul dari pembagian dengan 0
        df.fillna(0, inplace=True)


        # --- LANGKAH 2: PREPROCESSING UNTUK PREDIKSI ---
        cat_encoder = _classifier_artifacts["category_encoder"]
        scaler = _classifier_artifacts["scaler"]
        model = _classifier_artifacts["model"]

        # Transformasi kategori, handle kategori yang belum pernah dilihat
        df["category_name"] = df["category_name"].apply(lambda x: cat_encoder.transform([x])[0] if x in cat_encoder.classes_ else -1)

        # Filter baris dengan kategori yang tidak dikenal
        df_predictable = df[df["category_name"] != -1].copy()

        if df_predictable.empty:
            return "Tidak ada data kategori yang bisa diprediksi."

        # Pilih fitur yang sama dengan yang digunakan saat training
        features = [
            "category_name", "total_questions", "difficulty_score",
            "correct_ratio", "interaction", "difficulty_correct_ratio"
        ]
        # Ganti nama kolom 'category_name' menjadi 'category_name' untuk konsistensi dengan scaler
        df_predictable.rename(columns={'category_name': 'category_name'}, inplace=True)

        X = df_predictable[features]
        X_scaled = scaler.transform(X)

        # --- LANGKAH 3: PREDIKSI DAN PERSIAPAN PROMPT ---
        df_predictable["model_prediction"] = model.predict(X_scaled)

        # Ganti kembali nama kolom 'category_name' ke nama aslinya untuk output
        df_predictable['category_name_str'] = cat_encoder.inverse_transform(df_predictable['category_name'])


        # Siapkan data untuk prompt LLM
        performance_summary = df_predictable[["category_name_str", "accuracy", "model_prediction"]].to_dict("records")
        for record in performance_summary:
            record['category_name'] = record.pop('category_name_str') # Ganti nama kunci

        # Soal yang salah (logika ini bisa tetap sama)
        wrong_answers_list = []
        for item in performance_list:
            contoh_salah = item.get("example_incorrect_questions", [])
            for q in contoh_salah:
                wrong_answers_list.append({
                    "category_name": item["category_name"],
                    "question_text": q.get("question_text", ""),
                    "user_answer": q.get("user_answer", ""),
                    "correct_answer": q.get("correct_answer", "")
                })

        soal_salah_section = (
            "\n## ❌ Contoh Soal yang Salah:\n```json\n" + json.dumps(wrong_answers_list, indent=2) + "\n```"
            if wrong_answers_list else ""
        )

        # --- LANGKAH 4: PANGGIL LLM ---
        # Prompt untuk Mistral (bisa tetap sama)
        prompt = (
            'Kamu adalah "Reko", asisten AI pembelajaran yang cerdas, suportif, dan to the point.\n'
            'Tugasmu: bantu pengguna memahami kekuatan dan kelemahan belajar mereka berdasarkan data statistik dan prediksi model, serta contoh soal yang mereka jawab salah.\n\n'
            '## 📊 Ringkasan Performa + Prediksi Model Ensemble (Canggih):\n'
            '```json\n' + json.dumps(performance_summary, indent=2) + '\n```\n'
            f'{soal_salah_section}\n\n'
            '🎯 Tugasmu:\n'
            '- Sampaikan analisis menyeluruh tapi singkat dan natural berdasarkan data di atas.\n'
            '- Bahas 1-2 soal yang dijawab salah (jika ada) dan beri saran cara belajar yang relevan.\n'
            '- Berikan insight yang actionable dan spesifik berdasarkan kategori dan tingkat pemahaman yang diprediksi.\n'
            '- Tampilkan tabel markdown ringkas dan akhiri dengan motivasi ringan yang positif.'
        )

        # Kirim ke LLM (logika ini bisa tetap sama)
        api_key = os.environ.get("MISTRAL_API_KEY", "NsG05vqdKwNxJLnKw4SQW576qTf6YUbH") # Ganti dengan API key Anda jika perlu
        client = Mistral(api_key=api_key)
        response = client.chat.complete(
            model="mistral-small-latest",
            messages=[{"role": "user", "content": prompt}]
        )

        return response.choices[0].message.content

    except Exception as e:
        import traceback
        print(f"FATAL HYBRID ERROR: {e}\n{traceback.format_exc()}")
        return f"Terjadi error saat melakukan analisis hybrid: {e}"

# **ROUTING Flask + NGROK**

In [10]:
# --- ENDPOINT FLASK ---
app = Flask(__name__)

In [11]:
@app.route('/generate_recommendations_v2', methods=['POST'])
def generate_recommendations_endpoint_v2():
    """Endpoint untuk Model 1, sesuai dengan 'rekomendaimodels.py'."""
    try:
        data = request.get_json()
        if not data or "query_text_for_embedding" not in data or "search_keyword_for_wikipedia" not in data:
            return jsonify({"error": "query_text_for_embedding dan search_keyword_for_wikipedia wajib"}), 400

        query_embed = data["query_text_for_embedding"]
        search_wiki = data["search_keyword_for_wikipedia"]

        recommendations = get_dynamic_content_recommendations_colab(query_embed, search_wiki)
        return jsonify(recommendations)
    except Exception as e:
        print(f"API Error di /generate_recommendations_v2: {e}")
        return jsonify({"error": "Internal server error", "details": str(e)}), 500

In [12]:
@app.route('/holistic_hybrid_analysis', methods=['POST'])
def handle_holistic_hybrid_analysis():
    """
    Endpoint utama dan satu-satunya untuk Model 2.
    Menerima 'performance_data' dari aplikasi lokal.
    """
    print("COLAB SERVICE: Menerima permintaan untuk analisis hybrid...")
    data = request.json
    performance_list = data.get('performance_data')
    if not performance_list:
        return jsonify({"error": "Payload 'performance_data' dibutuhkan."}), 400

    summary_text = get_holistic_hybrid_analysis_from_data(performance_list)
    return jsonify({"holistic_analysis": summary_text})


In [13]:
if __name__ == '__main__':
    print("--- Menyiapkan Layanan AI Rekomind di Colab ---")
    # Muat Model 1
    _load_sentence_model_once_colab()

    # Muat Model 2 (Ensemble)
    _load_classifier_artifacts_once_colab()


--- Menyiapkan Layanan AI Rekomind di Colab ---
MODEL 1: Memuat model Sentence Transformer: 'paraphrase-multilingual-MiniLM-L12-v2'...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


MODEL 1: Model Sentence Transformer 'paraphrase-multilingual-MiniLM-L12-v2' berhasil dimuat.
MODEL 2: Memuat artefak untuk model Ensemble...
✅ MODEL 2: Artefak Ensemble (model, scaler, encoder) berhasil dimuat.


In [14]:
    def run_flask_app():
        # Menjalankan app Flask tanpa debug mode untuk produksi di thread
        app.run(host="0.0.0.0", port=5000, debug=False)

    flask_thread = threading.Thread(target=run_flask_app)
    flask_thread.daemon = True
    flask_thread.start()


In [15]:
    # Setup ngrok
    NGROK_AUTH_TOKEN = os.environ.get("NGROK_AUTH_TOKEN", "1wIKVgVrfDvzbOpqvTYd84P3qTo_649KyUcCJXPYtKcBbygkh")
    if NGROK_AUTH_TOKEN:
        conf.get_default().auth_token = NGROK_AUTH_TOKEN
    else:
        print("\n!!! PERINGATAN: NGROK_AUTH_TOKEN tidak diset di 'Secrets' Colab. Tunnel mungkin memiliki batasan waktu. !!!")

    public_url_global = None
    try:
        # Menutup tunnel yang mungkin sudah ada
        for tunnel in ngrok.get_tunnels():
            ngrok.disconnect(tunnel.public_url)

        # Membuat tunnel baru
        public_url_global = ngrok.connect(5000, "http", bind_tls=True)
        print("\n--- Aplikasi Siap Menerima Permintaan ---")
        print("-" * 50)
        print(f" * Ngrok Tunnel (HTTPS) Aktif di: {public_url_global}")
        print(f" * Endpoint Model 1: {public_url_global}/generate_recommendations_v2")
        print(f" * Endpoint Model 2 Hybrid: {public_url_global}/holistic_hybrid_analysis")
        print(" * API Rekomendasi di Colab SIAP DIAKSES.")
        print("-" * 50)

        # Menjaga script tetap berjalan
        while True:
            time.sleep(3600)
    except KeyboardInterrupt:
        print("\nCOLAB: Proses dihentikan oleh pengguna.")
    except Exception as e_ngrok:
        print(f"Error memulai Ngrok: {e_ngrok}")
    finally:
        print("COLAB: Menutup tunnel Ngrok...")
        if public_url_global:
            ngrok.disconnect(public_url_global)
        ngrok.kill()
        print("COLAB: Selesai.")

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit



--- Aplikasi Siap Menerima Permintaan ---
--------------------------------------------------
 * Ngrok Tunnel (HTTPS) Aktif di: NgrokTunnel: "https://e7c1-35-227-20-103.ngrok-free.app" -> "http://localhost:5000"
 * Endpoint Model 1: NgrokTunnel: "https://e7c1-35-227-20-103.ngrok-free.app" -> "http://localhost:5000"/generate_recommendations_v2
 * Endpoint Model 2 Hybrid: NgrokTunnel: "https://e7c1-35-227-20-103.ngrok-free.app" -> "http://localhost:5000"/holistic_hybrid_analysis
 * API Rekomendasi di Colab SIAP DIAKSES.
--------------------------------------------------
COLAB SERVICE: Menerima permintaan untuk analisis hybrid...

--- [HYBRID MODEL V2] Memulai Analisis dengan Ensemble Model ---


INFO:werkzeug:127.0.0.1 - - [04/Jul/2025 09:06:58] "POST /holistic_hybrid_analysis HTTP/1.1" 200 -


  MODEL 1: Mencari artikel Wikipedia untuk keyword: 'Bo Jackson which these celebrities Celebrities'
    -> MODEL 1: Artikel 'Janet Jackson' diproses (konten: 13301 chars).
    -> MODEL 1: Artikel 'Daftar penghargaan dan nominasi yang diterima oleh BTS' diproses (konten: 2091 chars).


...

    MODEL 1: Tidak ada hasil pencarian Wikipedia untuk 'Jon Jafari which these people Celebrities'.
  MODEL 1: Mencari artikel Wikipedia untuk keyword: 'Pneumonia what was the Celebrities'
    -> MODEL 1: Artikel 'George H. W. Bush' diproses (konten: 27841 chars).
    -> MODEL 1: Artikel 'Marlon Brando' diproses (konten: 95265 chars).


INFO:werkzeug:127.0.0.1 - - [04/Jul/2025 09:12:41] "POST /generate_recommendations_v2 HTTP/1.1" 200 -


COLAB SERVICE: Menerima permintaan untuk analisis hybrid...

--- [HYBRID MODEL V2] Memulai Analisis dengan Ensemble Model ---


INFO:werkzeug:127.0.0.1 - - [04/Jul/2025 09:12:52] "POST /holistic_hybrid_analysis HTTP/1.1" 200 -



COLAB: Proses dihentikan oleh pengguna.
COLAB: Menutup tunnel Ngrok...
COLAB: Selesai.
